### Python_Final_Project_Amruta_Joshi_1001792172
###### 1. Extract data from CNN website in a textfile
###### 1. Search for Most Actives tickers from text file
###### 2. Extract values for Prev Closed, Open and Volume values from Yahoo website for the most actives tickers
###### 3. Create a textfile and store values of Prev Closed, Open and Volume 
###### 4. Create a database in SQLite and a table with Ticker, Prev Closed, Open and Volume values

In [7]:
try:
    import re
    import requests
    myurl='https://money.cnn.com/data/hotstocks/'                # extract the ticker symbols
    hotstocks_handle = requests.get(myurl)
    hotstocks_text = hotstocks_handle.text
    fhand=open('Python_Final_Project.txt','w')                   # open text file in write mode
    for line in hotstocks_text:
        line.rstrip('\n')                                        # strip every line for space and new line character
        fhand.write(line)                                        # write data of whole page in to file
    fhand.close()
    hand=open('Python_Final_Project.txt')
    i=0
    for line in hand:
        line=line.strip()
        i+=1
        if re.search('^<h3>Most Actives</h3>',line):             # search for Most Actives in every line
            break
#     print(i)
    hand.close()
    hand=open('Python_Final_Project.txt')
    j=0
    for line in hand:
        line=line.strip()
        j+=1
        if re.search('^<h3>Gainers</h3>',line):                  # search for Gainers in every line
            break
#     print(j)
    hand.close()
    hand=open('Python_Final_Project.txt')
    k=0
    list_ticker=list()                                           # create list
    ticker_symbol_final_prev_close_dict=dict()                   # create dictionary
    ticker_symbol_final_Open_dict=dict()                         # create dictionary
    ticker_symbol_final_Volume_dict=dict()                       # create dictionary
    for line in hand:
        if i<k<j:
            line=line.strip()
            if line.startswith('<td><a href="/quote/quote.html?symb='):   # store data for every ticker from yahoo site in file
                ticker_symbol = re.findall('>.*?<', line)
                every_ticker=ticker_symbol[1].replace('>','').replace('<','')
                list_ticker.append(every_ticker)
                tickerurl = 'https://finance.yahoo.com/quote/{myticker}?p={myticker}&.tsrc=fin-srch-v1'
                mystock_handle=requests.get(tickerurl.format(myticker = every_ticker))
                mystock_text = mystock_handle.text
            
                ticker_symbol = re.findall('PREV_CLOSE-value.+?>.+?<', mystock_text) # get prev-close value of every ticker
                x=ticker_symbol[0]
                ticker_symbol_span= re.findall('span class.+?>.+?<',x)
                y=ticker_symbol_span[0]
                ticker_symbol_value= re.findall('>.+?<',y)
                ticker_symbol_final= ticker_symbol_value[0].replace('>','').replace('<','')
                ticker_symbol_final_prev_close_dict[every_ticker]=ticker_symbol_final
#             print(ticker_symbol_final)
                ticker_symbol_final_list=list(ticker_symbol_final_prev_close_dict.values())
    
                ticker_symbol_Open = re.findall('OPEN-value.+?>.+?<', mystock_text) # get open value of every ticker
                x=ticker_symbol_Open[0]
                ticker_symbol_span_Open= re.findall('span class.+?>.+?<',x)
                y=ticker_symbol_span_Open[0]
                ticker_symbol_value_Open= re.findall('>.+?<',y)
                ticker_symbol_final_Open= ticker_symbol_value_Open[0].replace('>','').replace('<','')
#                 print(ticker_symbol_final)
                ticker_symbol_final_Open_dict[every_ticker]=ticker_symbol_final_Open 
                ticker_symbol_final_Open_list=list(ticker_symbol_final_Open_dict.values())
    
                ticker_symbol_Volume = re.findall('TD_VOLUME-value.+?>.+?<', mystock_text) # get volume value of every ticker
                x=ticker_symbol_Volume[0]
                ticker_symbol_span_Volume= re.findall('span class.+?>.+?<',x)
                y=ticker_symbol_span_Volume[0]
                ticker_symbol_value_Volume= re.findall('>.+?<',y)
                ticker_symbol_final_Volume= ticker_symbol_value_Volume[0].replace('>','').replace('<','')
                ticker_symbol_final_Volume_noComma=ticker_symbol_final_Volume.replace(',','')
#                 print(ticker_symbol_final)
                ticker_symbol_final_Volume_dict[every_ticker]=ticker_symbol_final_Volume_noComma
                ticker_symbol_final_Volume_list=list(ticker_symbol_final_Volume_dict.values())
        
        k+=1
    hand.close()

    list_combo=[list_ticker,ticker_symbol_final_Open_list,ticker_symbol_final_list,ticker_symbol_final_Volume_list]
    with open("stocks.txt", "w") as file:                                    # write to text file in csv format
        file.write('TICKER, ''OPEN price, '+'PREV CLOSE price, '+'VOLUME\n')
        for x in zip(*list_combo):
            file.write("{0}, {1}, {2}, {3}\n".format(*x))
    import sqlite3                                                           # create a database in SQLite
    conn = sqlite3.connect('StocksDatabase.sqlite')
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS StocksTable')
    cur.execute('CREATE TABLE StocksTable (Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)')
    for i in range (0,10):                                                   # insert values for every ticker
        cur.execute('INSERT INTO StocksTable (Ticker, OpenPrice, PrevClose, Volume) VALUES (?,?,?,?)',(list_ticker[i],ticker_symbol_final_Open_list[i],ticker_symbol_final_list[i],ticker_symbol_final_Volume_list[i]))
        i+=1
    conn.commit()
    conn.close()
except:
    print('Web Site is down')                                               # show error if website is down